## Finding outliers in elections results

In [ ]:
import numpy as np, pandas as pd
pd.set_option('display.max_rows', 500)
df = pd.read_csv(r"elections_final.csv",encoding = "cp1255")


In [ ]:
print("num of kalpis: {}".format(len(df)))

### assign latin names to columns

In [ ]:
tav = df.columns[16]
naz = df.columns[27]
meretz = df.columns[25]
likud = df.columns[24]
liberman = df.columns[23]
benet = df.columns[26]
nun_cols = df.columns[[27,28,29,31]].tolist()
emet = df.columns[7]
gimel = df.columns[8]
elementar_cols = df.columns.tolist()[:7]
general_cols = df.columns.tolist()[:7]+[tav,benet,naz,emet,gimel]
parties_cols = df.columns[7:]
sums=df.loc[:,parties_cols].sum()
mask = sums>50000
bigs = sums[mask].index.tolist()
smalls = sums[~mask].index.tolist()



### First try: whether the votes for bennet have been assigned as a mistake to the nearby columns "naz".
The condition: naz is bigger than ten and is also bigger than Bennet

In [ ]:
df = pd.read_csv(r"elections_final.csv",encoding = "cp1255")
mask = (df[naz]>10)&(df[naz]>df[benet])
df.loc[mask,general_cols]


#### Very few: 3 of 10422. The analysts states: "Only Kibutz and arab vilages. Interesting, but no potential to the new right"
#### lets do it again with the results of the first day after the election

In [ ]:
df = pd.read_csv(r"elections_20190410.csv",encoding = "cp1255")
mask = (df[naz]>10)&(df[naz]>df[benet])
df.loc[mask,general_cols]
#df = pd.read_csv(r"elections_final.csv",encoding = "cp1255")

###### Afula and Givot Bar is probably a mistake (Sdom, not Helem). However, it has been fixed since 10/04/2019.

## Correlations -Second try
Lets utilize the linear correlation between "tav" and "bennet". In fact, lets look on the maximum correlation per party

In [ ]:
df = pd.read_csv(r"elections_final.csv",encoding = "cp1255")
corr=df.loc[:,bigs].corr()
mask = corr!=1
max_corr_party =corr[mask].idxmax()
max_corr_val = corr[mask].max()
corr_maxis = pd.concat([max_corr_party,max_corr_val],axis=1)
corr_maxis.columns = ["max_corr_party","max_corr_val"]
corr_maxis

#### o.k. Tav and New right are strongly correlated
#### Lets find places where tav is big and new right is small. It may be a mistake!

In [ ]:
df = pd.read_csv(r"elections_final.csv",encoding = "cp1255")
mask = (df[tav]>30)&(df[benet]<0.15*df[tav])
pd.set_option('display.max_rows', 500)
print("num of suspected calpis: {}".format(mask.sum()))
df.loc[mask,general_cols]

##### 151 calpis! However, it is easy to see that most, probably all, are from neighbourhood of concentrations of ultra orthodox (Hardal, haredi or Habad). Dividing the calpis to places we get:

In [ ]:
df.loc[mask,general_cols[0]].value_counts()

#### Third try: lets look whether one of the small parties (not only naz), get a significant percentage of the votes

### create perecantage dataframe

In [ ]:
total = df.loc[:,parties_cols].sum(axis=1)
fractions = (df.loc[:,parties_cols].values.T/total.values).T
fractions = pd.DataFrame(fractions,columns = parties_cols)
fractions_df = df.copy()
fractions_df.loc[:,parties_cols] = fractions.values

#### Looking for outliers: small party with high percentage 

In [ ]:
mask =(fractions_df.loc[:,smalls].max(axis=1)>0.05)
argmaxis=fractions_df.loc[mask,smalls].T.idxmax()
maxis=fractions_df.loc[mask,smalls].T.max()
maxis2 = df.loc[mask,smalls].T.max()
pd.set_option('display.max_rows', 500)
pd.concat([df.loc[mask,elementar_cols],argmaxis,maxis,maxis2,df.loc[mask,:].iloc[:,24]],axis=1)

#### These are not the places of new right. Lets look at the interesting calpi with index 6058

In [ ]:
df.iloc[6058,:]

#### BlueWhite is zero and a strange family whose letters are final caf and final nun got 91 votes. Thats a mistake, but the mistake is not important. Blue white cannot achieve or lose mandat in any case.